In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn  as sns


loan_file = pd.read_csv(".\\loan\\loan.csv")
## 1. Problem Statement ##

Problem to be solved: Identify patterns based on the columns which would be used to reject/deny loans, reduce the loan amount or charge higher interest rate.
## 2. Data Provided ##

The data shared is from a Consumer Finance company specializing in lending various types of loans. The data consist of 39717 rows spread across 111 columns. One of the columns indicates whether the loan is currently 

1. Ongoing
2. Fully Paid - Loan Fully paid by the Consumer
3. Charged Off - Consumer who has defaulted the loans and are marked for Collections by the Company.

loan_file.head(5)
loan_file.shape
# 3. Data Cleanup. #

#### 3.1 Identifying Null Columns ###

Based on cursory glance of the data, there are several columns which seem to have null values in the data. The first step is to identify colums which have null values.
type(loan_file.isnull().sum())

filtered_series = loan_file.isnull().sum()[loan_file.isnull().sum() == 39717]
print(filtered_series.shape)
filtered_series.to_csv('cols_having_empty_header.csv',header=['Count of Row Values'],index=True)

#### 3.2 Removing Null-valued columns ####

There are over 54 columns which seem to have null values. The first step is to remove these columns as they do not contribute to analysis.
Throughout the course of analysis,  a CSV file is generated to view the output and to assess the progress. 
loan_file.dropna(axis=1,how='all',inplace=True)
loan_file.shape
total_length = len(loan_file)

# loan_file.columns[]
print((loan_file.columns[(loan_file.isnull().sum()/total_length) > 0.75]).value_counts())
print((loan_file.columns[(loan_file.isnull().sum()/total_length) > 0.50]).value_counts())
print((loan_file.columns[(loan_file.isnull().sum()/total_length) > 0.25]).value_counts())

loan_file.drop(columns=loan_file.columns[(loan_file.isnull().sum()/total_length) > 0.50],inplace=True)
print(loan_file.shape)
loan_file.to_csv(".\\iteration_1.csv")
#### 3.3 Unique valued Columns ####

There are several columns which have unique values - these do not contribute to any analaysis due to invariant nature of data. The next step is to identify the list of columns and delete them.
unique_count_of_values = loan_file.nunique()
list_of_unique_count_of_values = unique_count_of_values[unique_count_of_values == 1].index.tolist()

print(list_of_unique_count_of_values)

# loan_file.drop(columns=['pymnt_plan','url','desc','initial_list_status','collections_12_mths_ex_med','policy_code','application_type','acc_now_delinq','chargeoff_within_12_mths','delinq_amnt','tax_liens'],inplace=True)


loan_file.drop(columns=list_of_unique_count_of_values,inplace=True)
print(loan_file.shape)

loan_file.to_csv(".\\iteration_2.csv")
#### 3.4 Removal of additional columns. ####

Upon visual analysis of columns, there are several columns which does not aid the analysis. Deleting rows which are descriptive and which does not contribute to identification.

| Field Name | Remarks |
| -------- | -------- |
| member_id | We shall retain id for reference purpose. Assumption - member_id will be retrieved from id |
| emp_title | Descriptive. Does not seem to contribute. |
| emp_length | Descriptive. Does not seem to contribute. |
| title | Descriptive. Does not seem to contribute. |
| last_credit_pull_d | The most recent month LC pulled credit for this loan - Does not contribute towards analysis.. |
| last_pymnt_amnt |  Last total payment amount received - Does not contribute towards analysis.|
| last_pymnt_d | Last month payment was received - Does not contribute towards analysis. |
| collection_recovery_fee | post charge off collection fee - Does not contribute towards analysis. |
| earliest_cr_line | The month the borrower's earliest reported credit line was opened - Does not contribute towards analysis.|
| addr_state | The state provided by the borrower in the loan application - All the states are allowed . |
| Zip Code| The first 3 numbers of the zip code provided by the borrower in the loan application.  Removed to prevent redlining and bias from the model. |
| issue_d |  The month which the loan was funded- Does not contribute towards analysis.|
| funded_amnt |  The total amount committed to that loan at that point in time.|
| funded_amnt_inv |  The total amount committed by investors for that loan at that point in time..|
| verification_status |  Indicates if income was verified by LC, not verified, or if the income source was verified.|
|url| URL for the LC page with listing data - Does not contribute to analysis|
|desc| Loan description provided by the borrower - Does not contribute to analysis|
|total_rec_late_fee|Late fees received to date - Does not contribute to analysis|
|total_rec_prncp|Principal received to date - Does not contribute to analysis|
|total_rec_int|Interest received to date - Does not contribute to analysis|
|out_prncp|Remaining outstanding principal for total amount funded - Does not contribute to analysis|
|out_prncp_inv|Remaining outstanding principal for portion of total amount funded by investors - Does not contribute to analysis|
|total_pymnt|Payments received to date for total amount funded - Does not contribute to analysis|
|total_pymnt_inv|Payments received to date for portion of total amount funded by investors - Does not contribute to analysis|






loan_file.drop(columns=['member_id','emp_title','emp_length','title','last_credit_pull_d','last_pymnt_amnt','last_pymnt_d','collection_recovery_fee','earliest_cr_line','addr_state','zip_code','issue_d','funded_amnt','funded_amnt_inv','verification_status','total_rec_late_fee','total_rec_prncp','total_rec_int','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','url','desc'],inplace=True)
print(loan_file.shape)
loan_file.to_csv(".\\iteration_3.csv")
# print(loan_file.tail(0))
#### 3.5 Remove In-Progress Loans from Analysis ####
Identify and remove entries which are ongoing. These will not contribute to the analysis.

At the end of this exercise, the data will contain only loans which are 'Charged Off' or 'Fully Paid'.
loan_file = loan_file[loan_file['loan_status'] != 'Current']
loan_file.head(5)
#### 3.6 Identify columns with empty vlaues and fill them. ####

From the remaining data, identify columns which have empty cells. The next step is to identify the appropriate data to be fill the values. For this, we shall use the mode to identify and populate the values.
print(loan_file.isna().sum() > 0)
print(loan_file['revol_util'].mode())
print(loan_file['pub_rec_bankruptcies'].mode())

loan_file['revol_util'].fillna(loan_file['revol_util'].mode()[0],inplace=True)
loan_file['pub_rec_bankruptcies'].fillna(loan_file['pub_rec_bankruptcies'].mode()[0])
loan_file['term'].value_counts()

loan_amnt_quantile = loan_file['loan_amnt'].quantile([0.5,0.75,0.8,.85,.9,.95,.99])

print(loan_amnt_quantile)
# 4. Visual Representation of Data #
We have now narrowed down to 21 columns from 111 columns. The next step to commence plotting data for exploratory analysis.
fig, ax = plt.subplots(figsize=(10,10))
sns.countplot(x='loan_status',data=loan_file)

total = float(len(loan_file))  # Total number of entries
for p in ax.patches:
    height = p.get_height()
    ax.text(
        p.get_x() + p.get_width() / 2.,
        height + 1.0,
        '{:.2f}%'.format((height / total) * 100),
        ha="center"
    )

plt.show()
From the above plot, there are about 14.59% of Consumers(excluding 'In-Progress' consumers) have defaulted(i.e. 'Charged Off'). The focus would now be to identify the distribution of data for 'Charged off' consumers to identify the pattern.
#### 4.1 : Segregating data based on analysis ####

The first step is to segregate the 'Charged off' consumers to identify patterns.
charged_off_file = loan_file[loan_file['loan_status']=='Charged Off']


charged_off_file['term'].apply(lambda x:x.replace('36 months','36'))
charged_off_file['term'].apply(lambda x:x.replace('60 months','60'))

# print(charged_off_file.head(2))

loan_file.to_csv(".\\iteration_4.csv")

#### 4.2 Plot of 'Charged off' consumers vs Grade/Sub-Grade

We now plot the graph of Count of 'Charged Off' consumers by Grade. 
# sns.boxplot(x='sub_grade',data=charged_off_file)
sns.set_palette("viridis")
sns.countplot(x='grade',data=charged_off_file,order=charged_off_file['grade'].value_counts().index)
plt.show()
Four of the grades show substantial charge-off's by customers. We shall now analyze the sub-grades of these 4 grades to identify patterns.
fig, ax = plt.subplots(figsize=(8,6))
sns.set_palette('colorblind')
# sns.countplot(x = 'grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] , hue = 'sub_grade',data =charged_off_file)
# sns.countplot(x = 'grade', order = ['B', 'C', 'D', 'E'] , hue = 'sub_grade',data =charged_off_file[charged_off_file['grade'].isin(['B','C','D','E'])])
sns.countplot(x='sub_grade',data=charged_off_file[charged_off_file['grade'].isin(['B'])])
plt.show()
fig, ax = plt.subplots(figsize=(8,6))
sns.set_palette('colorblind')
sns.countplot(x='sub_grade',data=charged_off_file[charged_off_file['grade'].isin(['C'])])
plt.show()
fig, ax = plt.subplots(figsize=(8,6))
sns.set_palette('colorblind')
sns.countplot(x='sub_grade',data=charged_off_file[charged_off_file['grade'].isin(['D'])])
plt.show()
fig, ax = plt.subplots(figsize=(8,6))
sns.set_palette('colorblind')
sns.countplot(x='sub_grade',data=charged_off_file[charged_off_file['grade'].isin(['E'])])
plt.show()

#### 4.3 Plot of Charge-off Consumers based on Home-ownership. ####

We now plot a graph based on home-ownership.
sns.countplot(x = 'home_ownership',data =charged_off_file, order=charged_off_file['home_ownership'].value_counts().index)
From the table above, consumers whose have rented their homes and are paying mortgage are more likely to defaut than other category.
#### 4.4 Plot of Charge-off Consumers based on Loan purpose. ####

Below is the plot of data based on Loan purpose.
fig, ax = plt.subplots(figsize=(30,10))
sns.set_palette('colorblind')
sns.countplot(x = 'purpose',data =charged_off_file, order=charged_off_file['purpose'].value_counts().index)
From the above data, people, who are planning to consolidate debt are more likely to default than other category.
#### 4.4 Plot of consumers who are deliquent for 2 years or more and who have had multiple inquiries in the last 6 months. ***

The below plot shows the accounts which are deliquent for 2 years or more or consumers who have made multiple enquiries. The notion is that account which are deliquent & consumers who have made multiple enquiries are in need of additional credit due to strained financial circumstances. 

However, the below plot shows that greater defaults have happened due to non-deliquent consumers. Also, there does not seem to be any relationship between inquiries and 'charge-off's.
fig, ax = plt.subplots(figsize=(20,20))
sns.set_palette('colorblind')
sns.countplot(x = 'delinq_2yrs',data =charged_off_file, order=charged_off_file['delinq_2yrs'].value_counts().index)
fig, ax = plt.subplots(figsize=(20,20))
sns.set_palette('colorblind')
sns.countplot(x = 'inq_last_6mths',data =charged_off_file, order=charged_off_file['inq_last_6mths'].value_counts().index)
#### 4.5 Plot of Open/Total Accounts in Charge-off #####


The plot of total number of accounts/number of open Accounts follow similar to normal curve with the peak of the charge off lying between 6 - 8 accounts.
# print(charged_off_file.info())

fig, ax = plt.subplots(figsize=(30,10))
sns.set_palette('colorblind')
# sns.countplot(x = 'open_acc',data =charged_off_file, order=charged_off_file['open_acc'].value_counts().index)
sns.countplot(x = 'open_acc',data =charged_off_file)


# list_openacc = lambda x: print(x['open_acc'],x['open_acc'].value_counts())

# list_openacc(charged_off_file)


fig, ax = plt.subplots(figsize=(30,10))
sns.set_palette('colorblind')
# sns.countplot(x = 'open_acc',data =charged_off_file, order=charged_off_file['open_acc'].value_counts().index)
sns.countplot(x = 'total_acc',data =charged_off_file)

#list_openacc = lambda x: print(x['total_acc'],x['total_acc'].value_counts())

# list_openacc(charged_off_file)

#### 4.6 Plot of Term vs Charged-off #####

Shorter Terms are more likely to default than longer terms, albeit very marginally.
# print(charged_off_file.info())

fig, ax = plt.subplots(figsize=(30,10))
sns.set_palette('colorblind')
# sns.countplot(x = 'open_acc',data =charged_off_file, order=charged_off_file['open_acc'].value_counts().index)
sns.countplot(x = 'term',data =charged_off_file)
# 5. Binning of continuous attributes #

For the purpose of the analysis, the following attributes would need to be 'binned' as discrete distribution. For this, we are binning the following attributes - Loan Amount, Interest Rate, Annual Income, DTI.

Instead of using mean/median & standard deviation, we have identified the minimum/maximum values and identify the bins.
print("Loan Amount:",charged_off_file['loan_amnt'].mean(),charged_off_file['loan_amnt'].median(),charged_off_file['loan_amnt'].max(),charged_off_file['loan_amnt'].min())
print("Interest Rate:",charged_off_file['int_rate'].max(),charged_off_file['int_rate'].min())
print("Annual Income:",charged_off_file['annual_inc'].mean(),charged_off_file['annual_inc'].median(),charged_off_file['annual_inc'].max(),charged_off_file['annual_inc'].min())
print("DTI:",charged_off_file['dti'].mean(),charged_off_file['dti'].median(),charged_off_file['dti'].max(),charged_off_file['dti'].min())
# print("Total Number of Accounts:",charged_off_file['total_acc'].median(),charged_off_file['total_acc'].max(),charged_off_file['total_acc'].min())

charged_off_file['dti_bin'] = pd.cut(charged_off_file['dti'], bins=6,precision =0,labels=['0-5','5-10','10-15','15-20','20-25','>25'])
charged_off_file['loan_amnt_bin'] = pd.cut(charged_off_file['loan_amnt'], bins=5,precision =0,labels=['0-7k','7k-14k','14k-21k','21k-28k','28k-35k'])
charged_off_file['annual_inc_bin'] = pd.cut(charged_off_file['annual_inc'], bins=9,precision =0,labels=['0-3k','3k-8k','8k-15k','15k-25k','25k-50k','50k-75k','75k-125k','125k-250k','250k-1250k'])
charged_off_file['total_acc_bin'] = pd.cut(charged_off_file['total_acc'], bins=8,precision =0,labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80'])
#charged_off_file['loan_amnt'] = pd.cut(charged_off_file['loan_amnt'], bins=10,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21
charged_off_file.to_csv(".\\iteration_5.csv")                                                                                                                                                                                        

#### 5.1 Analysis of dti Data ####

Based on analysis of dti bin data, dti value of 5-25 are more likely to default with dti ratio of 15-20 having max defaults.
fig, ax = plt.subplots(figsize=(15,10))
sns.set_palette('colorblind')
# sns.countplot(x = 'open_acc',data =charged_off_file, order=charged_off_file['open_acc'].value_counts().index)
sns.countplot(x = 'dti_bin',data =charged_off_file)



#### 5.2 Analysis of Loan Amount Data ####

Loan amount for charged off loans span between 900 - 35K. Analysis of loan data indicates many are smaller loan size, with more defaults happening between USD 0-7000 followed by 7K-14K and 14K-21K.
fig, ax = plt.subplots(figsize=(30,10))
sns.set_palette('colorblind')
# sns.countplot(x = 'open_acc',data =charged_off_file, order=charged_off_file['open_acc'].value_counts().index)
sns.countplot(x = 'loan_amnt_bin',data =charged_off_file)

loan_file['dti_bin'] = pd.cut(loan_file['dti'], bins=6,precision =0,labels=['0-5','5-10','10-15','15-20','20-25','>25'])
loan_file['loan_amnt_bin'] = pd.cut(loan_file['loan_amnt'], bins=5,precision =0,labels=['0-7k','7k-14k','14k-21k','21k-28k','28k-35k'])
loan_file['annual_inc_bin'] = pd.cut(loan_file['annual_inc'], bins=9,precision =0,labels=['0-3k','3k-8k','8k-15k','15k-25k','25k-50k','50k-75k','75k-125k','125k-250k','250k-1250k'])
loan_file['total_acc_bin'] = pd.cut(loan_file['total_acc'], bins=8,precision =0,labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80'])

plt.figure(figsize=(30,10))
sns.countplot(x = 'total_acc_bin',data =charged_off_file)
plt.show()